Coursera Capstone Week 3 Part 3

In [42]:
import numpy as np
import pandas as pd
import requests
import geocoder
import pickle
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json
from pandas.io.json import json_normalize 

Load Dataframe from Part 2

In [2]:
with open('dfpart2.pkl', 'rb') as handle:
    df = pickle.load(handle)
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Use geocoder library to get the latitude and longitude values of Toronto City.

In [10]:

lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario')
    lat_lng_coords = g.latlng

tor_lat =lat_lng_coords[0]
tor_long =lat_lng_coords[1]

#### Create a map of Toronto City with neighborhoods superimposed on top.

In [22]:
map_toronto = folium.Map(location=[tor_lat, tor_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [27]:
east_data = df[df['Borough']=='East Toronto'].reset_index(drop=True)
east_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558


Let's get the geographical coordinates of East Toronto.

In [25]:
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, East Toronto, Ontario')
    lat_lng_coords = g.latlng

east_lat =lat_lng_coords[0]
east_long =lat_lng_coords[1]

In [30]:
map_east = folium.Map(location=[east_lat, east_long], zoom_start=11)

# add markers to map
for lat, lng, label in zip(east_data['Latitude'], east_data['Longitude'], east_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_east)  
    
map_east

#### Define Foursquare Credentials and Version

In [31]:
CLIENT_ID = 'SCH5UTBS50JUZVK00TIXXAS2RD5J1CMZOFQSBUDICG35TX0Y' # your Foursquare ID
CLIENT_SECRET = 'WNGTUZCA4RPXDZGR3DNWIUWNNP3XLHRVZ2V3DCEHKS41B3Q4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SCH5UTBS50JUZVK00TIXXAS2RD5J1CMZOFQSBUDICG35TX0Y
CLIENT_SECRET:WNGTUZCA4RPXDZGR3DNWIUWNNP3XLHRVZ2V3DCEHKS41B3Q4


Let's explore the first neighborhood in our dataframe.

In [32]:
east_data.loc[0, 'Neighbourhood']

u'The Beaches'

In [34]:
neighborhood_latitude = east_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = east_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = east_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.6763574, -79.2930312.


#### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [36]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SCH5UTBS50JUZVK00TIXXAS2RD5J1CMZOFQSBUDICG35TX0Y&client_secret=WNGTUZCA4RPXDZGR3DNWIUWNNP3XLHRVZ2V3DCEHKS41B3Q4&v=20180605&ll=43.6763574,-79.2930312&radius=500&limit=100'

In [37]:
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5b9c198d4c1f670d9370f3df'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-4e77e3861f6ecf8d3648300c-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          u'suffix': u'.png'},
         u'id': u'4bf58dd8d48988d1e0931735',
         u'name': u'Coffee Shop',
         u'pluralName': u'Coffee Shops',
         u'primary': True,
         u'shortName': u'Coffee Shop'}],
       u'id': u'4e77e3861f6ecf8d3648300c',
       u'location': {u'address': u'637 Kingston Rd.',
        u'cc': u'CA',
        u'city': u'Toronto',
        u'country': u'Canada',
        u'crossStreet': u'at Main St.',
        u'distance': 486,
        u'formattedAddress': [u'637 Kingston Rd. (at Main St.)',
         u'Toronto ON M4

In [38]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.678798,-79.298045
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Upper Beaches,Neighborhood,43.680563,-79.292869
3,Beaches Fitness,Gym / Fitness Center,43.680319,-79.290991


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Manhattan

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
east_venues = getNearbyVenues(names=east_data['Neighbourhood'],
                                   latitudes=east_data['Latitude'],
                                   longitudes=east_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business reply mail Processing Centre969 Eastern


In [47]:
print(east_venues.shape)
east_venues.head()

(123, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Beaches Fitness,43.680319,-79.290991,Gym / Fitness Center
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [48]:
east_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business reply mail Processing Centre969 Eastern,20,20,20,20,20,20
Studio District,40,40,40,40,40,40
The Beaches,4,4,4,4,4,4
"The Beaches West, India Bazaar",17,17,17,17,17,17
"The Danforth West, Riverdale",42,42,42,42,42,42


In [50]:
print('There are {} uniques categories.'.format(len(east_venues['Venue Category'].unique())))

There are 68 uniques categories.


## 3. Analyze Each Neighborhood

In [53]:
# one hot encoding
east_onehot = pd.get_dummies(east_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
east_onehot['Neighbourhood'] = east_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [east_onehot.columns[-1]] + list(east_onehot.columns[:-1])
east_onehot = east_onehot[fixed_columns]

east_onehot.head()

,Neighbourhood,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
east_onehot.shape

(123, 69)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [70]:
east_grouped = east_onehot.groupby('Neighbourhood').mean().reset_index()
east_grouped

,Neighbourhood,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail,Yoga Studio
0,Business reply mail Processing Centre969 Eastern,0.00000,0.05,0.00000,0.000,0.000,0.000000,0.000,0.050000,0.00000,...,0.000000,0.000,0.05,0.05,0.05000,0.000,0.000000,0.000000,0.00000,0.05000
1,Studio District,0.05000,0.00,0.05000,0.025,0.025,0.025000,0.025,0.025000,0.00000,...,0.025000,0.025,0.00,0.00,0.00000,0.025,0.000000,0.000000,0.00000,0.02500
2,The Beaches,0.00000,0.00,0.00000,0.000,0.000,0.000000,0.000,0.000000,0.00000,...,0.000000,0.000,0.00,0.00,0.00000,0.000,0.000000,0.000000,0.00000,0.00000
3,"The Beaches West, India Bazaar",0.00000,0.00,0.00000,0.000,0.000,0.000000,0.000,0.058824,0.00000,...,0.058824,0.000,0.00,0.00,0.00000,0.000,0.058824,0.058824,0.00000,0.00000
4,"The Danforth West, Riverdale",0.02381,0.00,0.02381,0.000,0.000,0.047619,0.000,0.023810,0.02381,...,0.000000,0.000,0.00,0.00,0.02381,0.000,0.000000,0.000000,0.02381,0.02381


In [71]:
east_grouped.shape

(5, 69)

In [75]:
num_top_venues = 5

for hood in east_grouped['Neighbourhood']:
    #print("----"+hood+"----")
    temp = east_grouped[east_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

                venue  freq
0  Light Rail Station  0.10
1         Yoga Studio  0.05
2          Restaurant  0.05
3       Auto Workshop  0.05
4       Garden Center  0.05


                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2  American Restaurant  0.05
3   Italian Restaurant  0.05
4            Gastropub  0.05


                  venue  freq
0          Neighborhood  0.25
1  Gym / Fitness Center  0.25
2                   Pub  0.25
3           Coffee Shop  0.25
4    Light Rail Station  0.00


               venue  freq
0        Coffee Shop  0.06
1       Burger Joint  0.06
2  Fish & Chips Shop  0.06
3                Gym  0.06
4     Sandwich Place  0.06


                venue  freq
0    Greek Restaurant  0.26
1      Ice Cream Shop  0.07
2         Coffee Shop  0.07
3           Bookstore  0.05
4  Italian Restaurant  0.05




In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = east_grouped['Neighbourhood']

for ind in np.arange(east_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(east_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant
1,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Ice Cream Shop,Gym / Fitness Center,Fish Market,Coworking Space
2,The Beaches,Neighborhood,Coffee Shop,Gym / Fitness Center,Pub,Cosmetics Shop,Cocktail Bar,Comfort Food Restaurant,Comic Shop,Convenience Store,Coworking Space
3,"The Beaches West, India Bazaar",Fish & Chips Shop,Pub,Coffee Shop,Gym,Liquor Store,Fast Food Restaurant,Movie Theater,Burrito Place,Burger Joint,Park
4,"The Danforth West, Riverdale",Greek Restaurant,Ice Cream Shop,Coffee Shop,Italian Restaurant,Bookstore,Fruit & Vegetable Store,Indian Restaurant,Health Food Store,Grocery Store,Trail


## 4. Cluster Neighborhoods

In [97]:
# set number of clusters
kclusters = 3

east_grouped_clustering = east_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(east_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 0], dtype=int32)

In [98]:
toronto_merged = east_data
print toronto_merged
# add clustering labels
print kmeans.labels_
toronto_merged['Cluster Labels'] = kmeans.labels_
print toronto_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

  Postcode       Borough                                     Neighbourhood  \
0      M4E  East Toronto                                       The Beaches   
1      M4K  East Toronto                      The Danforth West, Riverdale   
2      M4L  East Toronto                    The Beaches West, India Bazaar   
3      M4M  East Toronto                                   Studio District   
4      M7Y  East Toronto  Business reply mail Processing Centre969 Eastern   

    Latitude  Longitude  Cluster Labels  
0  43.676357 -79.293031               3  
1  43.679557 -79.352188               2  
2  43.668999 -79.315572               1  
3  43.659526 -79.340923               4  
4  43.662744 -79.321558               0  
[2 2 1 2 0]
  Postcode       Borough                                     Neighbourhood  \
0      M4E  East Toronto                                       The Beaches   
1      M4K  East Toronto                      The Danforth West, Riverdale   
2      M4L  East Toronto         

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Neighborhood,Coffee Shop,Gym / Fitness Center,Pub,Cosmetics Shop,Cocktail Bar,Comfort Food Restaurant,Comic Shop,Convenience Store,Coworking Space
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Ice Cream Shop,Coffee Shop,Italian Restaurant,Bookstore,Fruit & Vegetable Store,Indian Restaurant,Health Food Store,Grocery Store,Trail
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Fish & Chips Shop,Pub,Coffee Shop,Gym,Liquor Store,Fast Food Restaurant,Movie Theater,Burrito Place,Burger Joint,Park
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Ice Cream Shop,Gym / Fitness Center,Fish Market,Coworking Space
4,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,0,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant


In [99]:
# create map
map_clusters = folium.Map(location=[east_lat, east_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Business reply mail Processing Centre969 Eastern,0,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant


In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"The Beaches West, India Bazaar",1,Fish & Chips Shop,Pub,Coffee Shop,Gym,Liquor Store,Fast Food Restaurant,Movie Theater,Burrito Place,Burger Joint,Park


In [102]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,2,Neighborhood,Coffee Shop,Gym / Fitness Center,Pub,Cosmetics Shop,Cocktail Bar,Comfort Food Restaurant,Comic Shop,Convenience Store,Coworking Space
1,"The Danforth West, Riverdale",2,Greek Restaurant,Ice Cream Shop,Coffee Shop,Italian Restaurant,Bookstore,Fruit & Vegetable Store,Indian Restaurant,Health Food Store,Grocery Store,Trail
3,Studio District,2,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Ice Cream Shop,Gym / Fitness Center,Fish Market,Coworking Space


In East Toronto 3 different clusters can be made. Beaches Danforth and Studio District are quire similar areas lying in the same cluster.